### --- Day 22: Monkey Market ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/22

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2024, 22);
WriteLines(inputLines);

Loading puzzle file: Day22.txt
Total lines: 1635
Max line length: 8

8379519
8825018
7346587
8726021
1408335


Ok, so the simple approach is to implement the mix / prune functions and generate, but I suspect this won't be enough for stage 2 :) Probably we need to cache the results or something like that, but we'll cross that bridge when we come to it.

In [4]:
using Num = uint;

In [5]:
// To mix a value into the secret number, calculate the bitwise XOR of the given
// value and the secret number. Then, the secret number becomes the result of that
// operation. (If the secret number is 42 and you were to mix 15 into the secret
// number, the secret number would become 37.)

Num Mix(Num a, Num b) => a ^ b;

// To prune the secret number, calculate the value of the secret number modulo
// 16777216. Then, the secret number becomes the result of that operation. (If the
// secret number is 100000000 and you were to prune the secret number, the secret
// number would become 16113920.)

Num Prune(Num a) => a % 16777216; // ie take first 24 bits


In [6]:
// In particular, each buyer's secret number evolves into the next secret number in the sequence via the following process:

Num Evolve(Num secret)
{
    // Note: this function will overflow a 32-bit integer. Technically it's OK
    // since the bits are lost from the top and the result is pruned to the 24
    // lower bits

    // checked {
        // Each step of the above process involves mixing and pruning:

        // Calculate the result of multiplying the secret number by 64. Then, mix
        // this result into the secret number. Finally, prune the secret number.
        secret = Prune(Mix(secret * 64, secret));

        // Calculate the result of dividing the secret number by 32. Round the
        // result down to the nearest integer. Then, mix this result into the secret
        // number. Finally, prune the secret number.
        secret = Prune(Mix(secret / 32, secret));

        // Calculate the result of multiplying the secret number by 2048. Then, mix
        // this result into the secret number. Finally, prune the secret number.
        secret = Prune(Mix(secret * 2048, secret));
    // }

    return secret;
}

In [7]:
// So, if a buyer had a secret number of 123, that buyer's next ten secret numbers would be:

// 15887950
// 16495136
// 527345
// 704524
// 1553684
// 12683156
// 11100544
// 12249484
// 7753432
// 5908254

Num testSecret = 123;
foreach (var s in Enumerable.Range(0, 10))
{
    testSecret = Evolve(testSecret);
    Console.WriteLine(testSecret);
}

15887950
16495136
527345
704524
1553684
12683156
11100544
12249484
7753432
5908254


In [8]:
Num Evolve2K(Num secret) => Enumerable.Range(0, 2000).Aggregate(secret, (s, i) => Evolve(s));

long EvolveSum(string[] inputLines) => inputLines.Select(Num.Parse)
                                        .Select(Evolve2K)
                                        .Sum(s => (long)s);

In [9]:
// For each buyer, simulate the creation of 2000 new secret numbers. What is the sum of the 2000th secret number generated by each buyer?

var part1Answer = EvolveSum(inputLines);
Console.WriteLine(part1Answer);

13753970725


In [10]:
// 13753970725 is correct
Ensure(13753970725, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/22

Ok, so for this one we have to find sequences of 4 price changes, and find the most common sequence such that we get the highest price occurring at the 4th element in the sequence.

This certainly feels sliding window-ish, but I don't think it quite is, since we not aggregating the window (eg rolling sum). We need to know the full sequence of 4 prices in each iteration. (Actually, we might be able to do something clever with bit-shifting, but maybe we'll try that later.) For now, if we model the sequence as a tuple, we can use them as the key in a dictionary, and take the entry with the highest sum of prices.

In [12]:
// Of course, the secret numbers aren't the prices each buyer is offering! That
// would be ridiculous. Instead, the prices the buyer offers are just the ones
// digit of each of their secret numbers.

IEnumerable<int> Evolve2(Num seed, int times = 2001)
{
    var i = 0;
    while (i++ < times)
    {
        yield return (int)(seed % 10);
        seed = Evolve(seed);
    }
}

In [13]:
// Unfortunately, the monkey only knows how to decide when to sell by looking at
// the changes in price. Specifically, the monkey will only look for a specific
// sequence of four consecutive changes in price, then immediately sell when it
// sees that sequence.

using DiffSequence = (int a, int b, int c, int d);

In [14]:
using DiffPrice = (DiffSequence sequence, int price);

IEnumerable<DiffPrice> GetDiffPrices(int[] seq)
{
    for (var i = 4; i < seq.Length; i++)
    {
        var (a, b, c, d, e) = (seq[i-4], seq[i-3], seq[i-2], seq[i-1], seq[i]);
        DiffPrice dp = ((b-a, c-b, d-c, e-d), seq[i]);
        yield return dp;
    }
}

// Each buyer only wants to buy one hiding spot, so after the hiding spot is
// sold, the monkey will move on to the next buyer.

IEnumerable<DiffPrice> FirstOnly(IEnumerable<DiffPrice> diffPrices)
{
    HashSet<DiffSequence> seen = new(2000);
    foreach (var dp in diffPrices.Where(x => !seen.Contains(x.sequence)))
    {
        yield return dp;
        seen.Add(dp.sequence);
    }
}

int GetMostBananas(string[] inputLines)
{
    var seeds = inputLines.Select(Num.Parse);
    
    var q = from seed in seeds
            let prices = Evolve2(seed).ToArray()
            from diffPrice in FirstOnly(GetDiffPrices(prices))
            group diffPrice by diffPrice.sequence into diffGroup
            let total = diffGroup.Sum(x => x.price)
            orderby total descending
            select (diffGroup.Key, total);

    var best = q.First();
    Console.WriteLine($"Best is {best}");
    return best.total;
}

In [15]:
// Suppose the initial secret number of each buyer is:

// 1
// 2
// 3
// 2024

// There are many sequences of four price changes you could tell the monkey, but
// for these four buyers, the sequence that will get you the most bananas is
// -2,1,-1,3. Using that sequence ... you would get 23 (7 + 7 + 9) bananas!

string[] testInputLines = 
[
    "1",
    "2",
    "3",
    "2024"
];

GetMostBananas(testInputLines);

Best is ((-2, 1, -1, 3), 23)


In [16]:
// Figure out the best sequence to tell the monkey so that by looking for that
// same sequence of changes in every buyer's future prices, you get the most
// bananas in total. What is the most bananas you can get?

var part2Answer = GetMostBananas(inputLines);

Best is ((-3, 0, 4, 0), 1570)


In [17]:
// 1570 is correct!
Ensure(1570, part2Answer);

#### Let's try the sliding window

Just for funsies, let's try the sliding window approach. Since each secret number is only a single digit, the diffs between secret numbers are small enough that we can fit a sequence of four into a single integer. So we can maintain the sliding window of price changes using bit shifting. Let's give it a go!

In [19]:
// Push 4 byte values into a single integer. This becomes our sequence
// "fingerprint"

int Push(int current, int value) => (current << 8) | (value & byte.MaxValue);

In [20]:
int GetInitialFingerprint(IEnumerable<int> items) => items.Aggregate(0, (current, next) => Push(current, next));
int GetNextFingerprint(int current, int remove, int add) => Push(current, add);

void GetMostBananas_Sliding(string[] inputLines)
{
    var seeds = inputLines.Select(Num.Parse);

    Dictionary<int, int> bananas = new(); // Bananas per sequence

    foreach (var seed in seeds)
    {
        var prices = Evolve2(seed).ToArray();
        var diffs = prices.Index().Skip(1).Select(x => x.Item - prices[x.Index - 1]);
        var fingerprints = SlidingWindow(diffs, 4, GetInitialFingerprint, GetNextFingerprint);

        HashSet<int> seen = new(2000);

        foreach (var (fingerprint, num) in fingerprints.Zip(prices[4..]))
        {
            if (!seen.Contains(fingerprint))
            {
                bananas[fingerprint] = bananas.TryGetValue(fingerprint, out var x) switch {
                    false => num,
                    true => x + num
                };
                seen.Add(fingerprint);
            }
        }
    }

    var best = bananas.Values.Max();
    Console.WriteLine($"Best is {best}");
}

GetMostBananas_Sliding(testInputLines);
GetMostBananas_Sliding(inputLines);

Best is 23
Best is 1570
